# load packages and functions

In [1]:
import re
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os 

def readAlignment(file):
    with open(file) as f:
        lines = f.readlines()
    line_numbers = [i for i in range(len(lines)) if '>' in lines[i] ]
    names = [lines[i].replace('\n','').replace('>','') for i in line_numbers]
    sequences = [''.join(lines[i+1:i+3]).replace('\n','') for i in line_numbers]
    msa = np.zeros([len(names),len(sequences[0])+1],dtype=np.object_)   
    msa[:,0] = names
    for tick_l in range(len(sequences)):
        msa[tick_l,1:] = list(sequences[tick_l])
    return msa

# INPUTS:
- **repeats_start**: where in the alignment the repeats start
- **r_len**: how many residues long are the repeats?

In [3]:
repeats_start = [28,39,57,68,86,97,108]
r_len = 11
full_length_file = '/home/as974/ada/repeatsProject/PvLEA4_full_b0.35/PvLEA4_full_b0.35.a2m'


# splicing the repeats

In [4]:
msa = readAlignment(full_length_file)
msa_df = pd.DataFrame(msa,columns = ['name']+list(map(str,np.arange(1,144)))) 
msa_df.head()

,name,1,2,3,4,5,6,7,8,9,...,134,135,136,137,138,139,140,141,142,143
0,S6BNK7/1-143,m,v,k,q,d,n,l,d,q,...,m,k,s,d,k,n,k,n,c,c
1,UniRef100_A0A2P4V0U1/16-133,.,.,.,.,.,.,.,k,n,...,.,.,.,.,.,.,.,.,.,.
2,UniRef100_A0A2P4V0U1/200-313,.,.,.,.,.,.,.,.,.,...,.,.,.,.,.,.,.,.,.,.
3,UniRef100_A0A2P4V0U1/304-437,.,.,.,.,.,.,y,n,q,...,v,e,d,t,k,d,k,.,.,.
4,UniRef100_A0A2P4V0U1/497-629,.,.,.,.,.,.,y,n,q,...,a,e,d,a,k,h,.,.,.,.


In [5]:
msa_df[['start','end']] = msa_df.name.str.split('/',expand = True)[1].str.split('-',expand=True)
msa_df['name'] = msa_df.name.str.split('/',expand=True)[0]
msa_df.head()

In [8]:
repeats_msa_df = pd.DataFrame(columns=['name','start','end'] + list(map(str,np.arange(1,r_len+1))))
for index,row in msa_df.iterrows():
    for rs in repeats_start:
        rep1 = row[['name'] + list(map(str,np.arange(rs,rs+r_len)))]
        rep1['start'] = str(int(row['start']) + rs-1)
        rep1['end'] = str(int(row['start']) + rs+r_len-1)
        rep1.rename(dict(zip(list(map(str,np.arange(rs,rs+r_len))),list(map(str,np.arange(1,r_len+1))))), inplace=True)
        repeats_msa_df = repeats_msa_df.append(rep1,ignore_index=True)

In [9]:
# I felt like the '-' were messing up the parsing in plm.c 
for index,i in enumerate(repeats_msa_df.name):
    if '-' in i:
        print(index,i)

15890 UniRef100_P0C5A4-2
15891 UniRef100_P0C5A4-2
15892 UniRef100_P0C5A4-2
15893 UniRef100_P0C5A4-2
15894 UniRef100_P0C5A4-2
15895 UniRef100_P0C5A4-2
15896 UniRef100_P0C5A4-2


In [10]:
repeats_msa_df.name = repeats_msa_df.name.str.replace('-','_')

In [12]:
# removing the existing files
try:
    os.mkdir('/home/as974/ada/repeatsProject/PvLEA4_full_b0.35/pyOutput')
except:
    print('pyOutput file already exists')
try: 
    os.remove('/home/as974/ada/repeatsProject/PvLEA4_full_b0.35/PvLEA4.a2m')
    print('removed!')
except: print('never existed!')

pyOutput file already exists
removed that shit!


In [13]:
with open('/home/as974/ada/repeatsProject/PvLEA4_full_b0.35/PvLEA4.a2m','w') as file:
    for index,row in repeats_msa_df.iterrows():
        c = row.values
        file.writelines('>' + c[0] + '/' + c[1] + '-' + c[2] + '\n' + ''.join(c[3:]) + '\n')                   

# Don't have to run - just checking things

In [11]:
# for index,i in enumerate(repeats_msa_df.name):
#     if '-' in i:
#         print(index,i)